# Survival analysis with Catboost

Unlike common supervised regression task where target variable is known and observed during the entire period in training dataset, survival regression has some limitations in terms of partially observed or **censored** target variable.  
**Censoring** is a form of missing data problem in which time to event is not observed for reasons such as termination of study before all recruited subjects have shown the event of interest or the subject has left the study prior to experiencing an event.  
  
Censoring types:
 * **Right-censored:** When the patient is lost to followup or the event does not occur within the study duration. 
 * **Left-censored:** When the patient had been on risk for disease for a period before entering the study.
 * **Interval-censored:** When time to event may be known only up to a time interval.
 * **Uncensored:** When everything is alright:)

## Cox proportional model

Survival models of this type can be viewed as consisting of two parts: the underlying <a href="https://en.wikipedia.org/wiki/Hazard_function"> baseline hazard </a> function, often denoted ${\displaystyle \lambda _{0}(t)}$, describing how the risk of event per time unit changes over time at baseline levels of covariates; and the effect parameters, describing how the hazard varies in response to explanatory covariates. 

Let $X_i=(X_{i1},...,X_{ip})$ be the realized values of the covariates for subject i. The hazard function for the Cox proportional hazards model has the form:

$$\lambda(t|X_i)=\lambda_0(t)exp(h(X_i)),$$
where $h(X_i)$ is the hazard rate for considered sample $X_i$, represented as CatBoost model.This expression gives the hazard function at time $t$ for subject $i$ with covariate vector (explanatory variables) $X_i$.The likelihood of the event to be observed occurring for subject $i$ at time $Y_i$ can be written as:

$$L_i(\beta)=
    \frac{\lambda(Y_i|X_i)}{\sum_{j: Y_j \ge Y_i}\lambda(Y_i|X_j)} =
    \frac{\lambda_0(Y_i)exp(h(X_i))}{\sum_{j: Y_j \ge Y_i}\lambda_0(Y_i)exp(h(X_j))} =
    \frac{exp(h(X_i))}{\sum_{j: Y_j \ge Y_i}exp(h(X_j))}
$$

Treating the subjects as if they were statistically independent of each other, the joint probability of all realized events is the following partial likelihood, where the occurrence of the event is indicated by $C_i = 1$:

$${\displaystyle L(\beta )=\prod _{i:C_{i}=1}L_{i}(\beta ).}$$

The corresponding log partial likelihood is:

$$\ell(\beta)=\sum_{i:C_i=1}(h(X_i) - log \sum_{j:Y_j \ge Y_i} exp(h(X_j)))$$

This is the function we are trying to optimize via CatBoost.

### Fit model 

In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sksurv.datasets import load_flchain

In [39]:
data, target = load_flchain()

In [40]:
target = pd.DataFrame(target)
target['label'] = np.where(target['death'], target['futime'], - target['futime'])

data = data.join(target)
data.head(10)

,age,chapter,creatinine,flc.grp,kappa,lambda,mgus,sample.yr,sex,death,futime,label
0,97.0,Circulatory,1.7,10,5.700,4.860,no,1997,F,True,85.0,85.0
1,92.0,Neoplasms,0.9,1,0.870,0.683,no,2000,F,True,1281.0,1281.0
2,94.0,Circulatory,1.4,10,4.360,3.850,no,1997,F,True,69.0,69.0
3,92.0,Circulatory,1.0,9,2.420,2.220,no,1996,F,True,115.0,115.0
4,93.0,Circulatory,1.1,6,1.320,1.690,no,1996,F,True,1039.0,1039.0
5,90.0,Mental,1.0,9,2.010,1.860,no,1997,F,True,1355.0,1355.0
6,90.0,Mental,0.8,1,0.430,0.880,no,1996,F,True,2851.0,2851.0
7,90.0,Nervous,1.2,10,2.470,2.700,no,1999,F,True,372.0,372.0
8,93.0,Respiratory,1.2,9,1.910,2.180,no,1996,F,True,3309.0,3309.0
9,91.0,Circulatory,0.8,6,0.791,2.220,no,1996,F,True,1326.0,1326.0


In [77]:
train, test = train_test_split(data)

In [86]:
features = ['age', 'chapter', 'creatinine', 'flc.grp', 'kappa', 'lambda', 'mgus', 'sample.yr', 'sex']
cat_features = ['sex', 'mgus', 'chapter', 'flc.grp', 'sample.yr']

In [79]:
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split

In [80]:
vals = train.mode().iloc[0]

In [81]:
train = train.fillna(vals)
test = test.fillna(vals)

In [87]:
train_pool = Pool(train[features], label=train['label'], cat_features=cat_features)
test_pool = Pool(test[features], label=test['label'], cat_features=cat_features)

In [172]:
model = CatBoostRegressor(iterations=100,
                          loss_function='Cox',
                          eval_metric='Cox',
                          metric_period=50
                         )


In [173]:
model.fit(train_pool, eval_set=test_pool)

0:	learn: 13717.0120433	test: 3905.4239773	best: 3905.4239773 (0)	total: 6.79ms	remaining: 672ms
50:	learn: 13384.7022346	test: 3804.2275700	best: 3804.2275700 (50)	total: 249ms	remaining: 239ms
99:	learn: 13124.4787555	test: 3723.5118027	best: 3723.5118027 (99)	total: 475ms	remaining: 0us

bestTest = 3723.511803
bestIteration = 99



### Evaluate metric 

In [174]:
# !pip install lifelines
from lifelines.utils import concordance_index

In [175]:
train_y_pred = model.predict(train_pool)
test_y_pred = model.predict(test_pool)

ci_train = concordance_index(train['futime'], -train_y_pred, train['death'])
ci_test = concordance_index(test['futime'], -test_y_pred, test['death'])
print(f'Train set. Concordance index:{ci_train:0.5f}')
print(f'Test set. Concordance index:{ci_test:0.5f}')


Train set. Concordance index:0.84141
Test set. Concordance index:0.83833


### Accelerated Failure Time (AFT) model 

Base parametric model  
$$\log T_i = \langle w, x_i \rangle + \epsilon_i$$  
$T_i$ - survival time of the i-th subject
$w$ - learnable parameters  
$x_i$ - subject time-invariant feature description  
$\epsilon_i$ - error term with. $\epsilon_i \sim \sigma Z_i$ where $Z_i$ is a prespecified a distribution and $\sigma$ is a scaling hyperparamater.

Idea - replace linear term $\langle w, x \rangle$ with boosted decision trees classifier $H(x)$.

### Fitting process 

As $T_i$ is random variable, we can utilize common approach - maximize likelihood function across $n$ observed subjects $$L= \prod_{i=1}^n f_T(t=T_i) \sim \sum_{i=1}^n \log f_T(t=T_i)$$ where $f_T(t)$ is a PDF of $T = g(\epsilon) = e^{H(x)+\epsilon}$ prespecified distribution's PDF.  
With an eye on interval form of censored target, likelihood function can be decomposed into two parts:   
a) likelihood for uncensored data  
b) likelihood for arbitrary censored data  
$$\log L = \sum_{i=1}^n \log f_T(t=T_i) = \sum_{a} \log f_T(t=T_a) + \sum_{b} \log (F_T(t=\overline{T_b})-F_T(t=\underline{T_{b}}))$$
where $F_T(t)$ is a prespecified distribution's CDF  
$\overline{T}, \underline{T}$ - upper and lower bounds of the interval  


In Catboost Z can be distributed according to one of three distributions:

| Distribution | CDF | PDF |
| --- | --- | --- |
| Normal |$$\dfrac{1}{2}(1+erf( \dfrac{z}{\sqrt{2}})) $$ | $$\dfrac{e^{\dfrac{-z^2}{2}}}{\sqrt{2\pi}}$$|
| Logistic | $$\dfrac{e^z}{1+e^z}$$| $$\dfrac{e^z}{(1+e^z)^2}$$ |
| Extreme | $$1-e^{-e^z}$$  | $$e^ze^{-e^z}$$  |

In order to use censored data in Catboost one should express target variable in interval form:
 * **Right-censored:** $[ A, +\infty ]$
 * **Left-censored:** $[0, B]$
 * **Interval-censored:** $[A, B]$
 * **Uncensored:** $[A, A]$  
where $A$, $B$ - finite non-negative numbers

### Fit model 

#### Load dataset 

We are going to use [WHAS500* dataset](https://github.com/sebp/scikit-survival/tree/master/sksurv/datasets/data) and will try to predict patients' lifetime after the hospital admission.  
*Table 1.2
of Hosmer, D.W. and Lemeshow, S. and May, S. (2008) Applied Survival
Analysis: Regression Modeling of Time to Event Data: Second Edition,
John Wiley and Sons Inc., New York, NY

In [3]:
import pandas as pd
import numpy as np
from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import train_test_split
from sksurv.datasets import load_whas500

In [4]:
data, target = load_whas500()
data = data.join(pd.DataFrame(target))

In order to use right-censored data in catboost use -1 to express $+\infty$. 

In [5]:
data['y_lower'] = data['lenfol']
data['y_upper'] = np.where(data['fstat'], data['lenfol'], -1)

In [6]:
stratifying_column = data['fstat']
data = data.drop(['fstat','lenfol'],axis=1)

In [9]:
train, test = train_test_split(data, test_size=0.2, stratify=stratifying_column, random_state=32)

In [10]:
data.head(5)

,afb,age,av3,bmi,chf,cvd,diasbp,gender,hr,los,miord,mitype,sho,sysbp,y_lower,y_upper
0,1,83.0,0,25.54051,0,1,78.0,0,89.0,5.0,1,0,0,152.0,2178.0,-1.0
1,0,49.0,0,24.02398,0,1,60.0,0,84.0,5.0,0,1,0,120.0,2172.0,-1.0
2,0,70.0,0,22.14290,0,0,88.0,1,83.0,5.0,0,1,0,147.0,2190.0,-1.0
3,0,70.0,0,26.63187,1,1,76.0,0,65.0,10.0,0,1,0,123.0,297.0,297.0
4,0,70.0,0,24.41255,0,1,85.0,0,63.0,6.0,0,1,0,135.0,2131.0,-1.0


In [11]:
features = data.columns.difference(['y_lower', 'y_upper'], sort=False)
cat_features = ['cvd','afb','sho','chf','av3', 'miord', 'mitype', 'gender']

In [12]:
train_pool = Pool(train[features], label=train.loc[:,['y_lower','y_upper']], cat_features=cat_features)
test_pool = Pool(test[features], label=test.loc[:,['y_lower','y_upper']], cat_features=cat_features)

loss function format: SurvivalAft:dist={distname};scale={int}  
Hyperparameters:
* distname $\in$ [Normal, Logistic, Extreme]
* scale - any positive number representing distribution scale

eval metric: SurvivalAft   
Metric is calculated as "interval MAE":  
$$\text{Interval MAE} = \begin{cases}
0 & \text{if $\hat{y} \in [\underline{y}, \overline{y}]$}\\
\min{(|\hat{y}-\underline{y}|,|\hat{y}-\overline{y}|)} & \text{if $\hat{y} \notin [\underline{y}, \overline{y}]$}
\end{cases}$$


In [13]:
model_normal = CatBoostRegressor(iterations=500,
                                 loss_function='SurvivalAft:dist=Normal',
                                 eval_metric='SurvivalAft',
                                 verbose=0
                                )

In [14]:
model_logistic = CatBoostRegressor(iterations=500,
                                 loss_function='SurvivalAft:dist=Logistic;scale=1.2',
                                 eval_metric='SurvivalAft',
                                 verbose=0)

In [15]:
model_extreme = CatBoostRegressor(iterations=500,
                                 loss_function='SurvivalAft:dist=Extreme;scale=2',
                                 eval_metric='SurvivalAft',
                                 verbose=0)

In [16]:
_ = model_normal.fit(train_pool, eval_set=test_pool)
_ = model_logistic.fit(train_pool, eval_set=test_pool)
_ = model_extreme.fit(train_pool, eval_set=test_pool)

In [17]:
train_predictions = pd.DataFrame({'y_lower': train['y_lower'],
                                  'y_upper': train['y_upper'],
                                  'preds_normal': model_normal.predict(train_pool, prediction_type='Exponent'),
                                  'preds_logistic': model_logistic.predict(train_pool, prediction_type='Exponent'),
                                  'preds_extreme': model_extreme.predict(train_pool, prediction_type='Exponent')})
train_predictions['y_upper'] = np.where(train_predictions['y_upper']==-1, np.inf, train_predictions['y_upper'])

test_predictions = pd.DataFrame({'y_lower': test['y_lower'],
                                  'y_upper': test['y_upper'],
                                  'preds_normal': model_normal.predict(test_pool, prediction_type='Exponent'),
                                  'preds_logistic': model_logistic.predict(test_pool, prediction_type='Exponent'),
                                  'preds_extreme': model_extreme.predict(test_pool, prediction_type='Exponent')})
test_predictions['y_upper'] = np.where(test_predictions['y_upper']==-1, np.inf, test_predictions['y_upper'])

### Evaluate metric 

In [18]:
def interval_mae(y_true_lower, y_true_upper, y_pred):
    mae = np.where((y_true_lower <= y_pred) & (y_pred <= y_true_upper),
                   0,
                   np.minimum(np.abs(y_true_lower-y_pred),
                              np.abs(y_true_upper-y_pred))) 
    return mae.mean()

In [19]:
distributions = ['normal', 'logistic', 'extreme']
print('Interval MAE')
for dist in distributions:
    train_metric = interval_mae(train_predictions['y_lower'], train_predictions['y_upper'], train_predictions[f'preds_{dist}'])
    test_metric = interval_mae(test_predictions['y_lower'], test_predictions['y_upper'], test_predictions[f'preds_{dist}'])
    print(f'Train set. dist:{dist}: {train_metric:0.2f}')
    print(f'Test set. dist:{dist}: {test_metric:0.2f}')
    print('---------------------------')

Interval MAE
Train set. dist:normal: 197.36
Test set. dist:normal: 348.89
---------------------------
Train set. dist:logistic: 401.47
Test set. dist:logistic: 477.54
---------------------------
Train set. dist:extreme: 324.11
Test set. dist:extreme: 307.46
---------------------------
